# 3.6 softmax回归的从零开始实现

In [2]:
#使用Fashion-MNIST数据集，并设置数据迭代器的批量大小为256.
import torch
from IPython import display
from d2l import torch as d2l

batch_size = 256
train_iter, test_iter = d2l.load_data_fashion_mnist(batch_size)

  0%|          | 0/26421880 [00:00<?, ?it/s]

Extracting ../data/FashionMNIST/raw/train-images-idx3-ubyte.gz to ../data/FashionMNIST/raw



  0%|          | 0/29515 [00:00<?, ?it/s]

Extracting ../data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to ../data/FashionMNIST/raw



  0%|          | 0/4422102 [00:00<?, ?it/s]

Extracting ../data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to ../data/FashionMNIST/raw



  0%|          | 0/5148 [00:00<?, ?it/s]

Extracting ../data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to ../data/FashionMNIST/raw



## 3.6.1 初始化模型参数

In [3]:
#这里的每一个样本都将用固定长度的向量表示。原始数据集中的每个样本都是28*28的图像，在本节中我们将展开每个图像，看作是长度为784的向量。
#在softmax回归中，我们的输出与类别一样多，数据集有10个类别，所以网络的输出维度为10.
#权重将构成一个784*10的矩阵，偏置将构成一个1*10的行向量，与线性回归一样，使用正态分布初始化权重w，偏置初始化为0.
num_inputs = 784
num_outputs = 10

w = torch.normal(0, 0.01, size=(num_inputs, num_outputs), requires_grad=True)
b = torch.zeros(num_outputs, requires_grad=True)

## 3.6.2 定义softmax操作

In [5]:
#回顾一下sum运算符如何沿着张量中的特定维度工作。
X = torch.tensor([[1.0, 2.0, 3.0], [4.0, 5.0, 6.0]])
X, X.sum(0, keepdims=True), X.sum(1, keepdims=True)

(tensor([[1., 2., 3.],
         [4., 5., 6.]]),
 tensor([[5., 7., 9.]]),
 tensor([[ 6.],
         [15.]]))

In [6]:
#回想实现softmax的三个步骤：
#1.对每个项求幂（使用exp）
#2.对每一行求和（小批量中每个样本是一行），得到每个样本的规范化常数
#3.将每一行处以其规范化常数，确保结果和为1
#分母或规范化常数，有时也称为配分函数（其对数称为对数-配分函数）

def softmax(X):
    X_exp = torch.exp(X)
    partition = X_exp.sum(1, keepdim=True)
    return X_exp / partition # 这里应用了广播机制

In [7]:
#对于任何随机出入，我们将元素变成一个非负数，依据概率原理，每行总和为1.
X = torch.normal(0, 1, (2, 5))
X_prob = softmax(X)
X_prob, X_prob.sum(1)

(tensor([[0.0469, 0.1339, 0.0771, 0.3101, 0.4320],
         [0.2606, 0.1128, 0.2011, 0.0898, 0.3357]]),
 tensor([1.0000, 1.0000]))

## 3.6.3 定义模型

In [8]:
#下面的代码定义了如何通过网络映射到输出。注意，将数据传递到模型之前，使用reshape函数将每张原始图像展平为向量。
def nex(X):
    return softmax(torch.matmul(X.reshape((-1, W.shape[0])), W) + b)

## 3.6.4定义损失函数

In [11]:
#实现交叉熵损失函数，交叉熵采用真实标签的预测概率的负对数似然。
#创建一个数据样本y_hat，其中包含2个样本在3个类别的预测概率，以及他们对应的标签y。
#有了y我们知道，在第一个样本中第一类是正确的预测；而在第二个样本中，第三类是正确的预测。
#使用y作为y_hat中概率的索引，我们选择第一个样本中第一类的概率和第二个样本中第三个类的概率。

y = torch.tensor([0, 2])
y_hat = torch.tensor([[0.1, 0.3, 0.6], [0.3, 0.2, 0.5]])
y_hat[[0, 1], y]

tensor([0.1000, 0.5000])

In [12]:
#现在只需1行代码就可以实现交叉熵损失函数
def cross_entropy(y_hat, y):
    return -torch.log(y_hat[range(len(y_hat)), y])
cross_entropy(y_hat, y)

tensor([2.3026, 0.6931])

## 3.6.5 分类精度